In [1]:
# set up
import os
import time
import numpy as np
import pandas as pd
import functools as func
import functions_v3 as f
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
home_path = "C:\\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\\07_Arbeitsordner\\04_Ergebnisse\\20220920_Ergebnisse_4_Wochen"
os.chdir(home_path)

#### KPI 4: CO2 emissions
---
Combined data frame for every entity in use case with:
- grey_energy [kWh] -> to be multiplied by co2 emission factor
- grey_energy [%] -> share grey energy consumed (1 - share renewable)

In [2]:
os.chdir(home_path)
os.chdir('Base_Case/')
use_case_home_path = os.getcwd()
start = time.time()
df = f.co2_emissions(use_case_home_path)
print(f'time: {round(time.time() - start, 2)}')
df

time: 99.93


,region-1-ec0-house-wind_grey_energy [kWh],region-1-ec0-house-wind_grey_energy [%],region-1-ec0-house-wind_region-1-id-1-wind-feedin_grey_energy [kWh],region-1-ec0-house-wind_region-1-id-1-wind-feedin_grey_energy [%],region-1-ec1-house-0_grey_energy [kWh],region-1-ec1-house-0_grey_energy [%],region-1-ec1-house-1_grey_energy [kWh],region-1-ec1-house-1_grey_energy [%],region-1-ec1-house-10_grey_energy [kWh],region-1-ec1-house-10_grey_energy [%],...,region-6-ec4-house-8_region-6-id-511-hh-2-household-electric-load_grey_energy [kWh],region-6-ec4-house-8_region-6-id-511-hh-2-household-electric-load_grey_energy [%],region-6-ec4-house-8_region-6-id-78-heatpump-loadprofile_grey_energy [kWh],region-6-ec4-house-8_region-6-id-78-heatpump-loadprofile_grey_energy [%],region-6-ec4-house-9_region-6-id-512-hh-2-household-electric-load_grey_energy [kWh],region-6-ec4-house-9_region-6-id-512-hh-2-household-electric-load_grey_energy [%],region-6-ec5-house-industry_grey_energy [kWh],region-6-ec5-house-industry_grey_energy [%],region-6-industry-electric-load_grey_energy [kWh],region-6-industry-electric-load_grey_energy [%]
2030-01-01T00:00,-126.140086,0.0,-0.0,0.0,0.607733,0.622659,0.611739,0.622659,0.706933,0.622659,...,1.098400,1.0,3.129416,1.0,1.098400,1.0,64.715234,0.622659,64.715234,1.0
2030-01-01T01:00,-121.633744,0.0,-0.0,0.0,0.608267,0.638371,0.625584,0.638371,0.666133,0.638371,...,1.038667,1.0,3.120415,1.0,1.038667,1.0,62.774245,0.638371,62.774245,1.0
2030-01-01T02:00,-119.323853,0.0,-0.0,0.0,0.655733,0.631698,0.667720,0.631698,0.712000,0.631698,...,1.055733,1.0,3.112027,1.0,1.055733,1.0,63.878920,0.631698,63.878920,1.0
2030-01-01T03:00,-119.464280,0.0,-0.0,0.0,0.675467,0.648326,0.680963,0.648326,0.660000,0.648326,...,1.932000,1.0,3.104372,1.0,1.932000,1.0,64.737669,0.648326,64.737669,1.0
2030-01-01T04:00,-122.984529,0.0,-0.0,0.0,0.583467,0.678785,0.587924,0.678785,1.209867,0.678785,...,1.860000,1.0,3.097578,1.0,1.860000,1.0,65.167316,0.678785,65.167316,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-10-07T19:00,-230.799438,0.0,-0.0,0.0,1.206133,0.138386,1.305892,0.138386,2.069333,0.138386,...,1.088000,1.0,2.841004,1.0,1.088000,1.0,60.644546,0.138386,60.644546,1.0
2030-10-07T20:00,-224.197998,0.0,-0.0,0.0,1.107467,0.120366,1.190875,0.120366,1.376533,0.120366,...,1.204000,1.0,2.831267,1.0,1.204000,1.0,60.003041,0.120366,60.003041,1.0
2030-10-07T21:00,-214.679024,0.0,-0.0,0.0,0.901067,0.174200,0.970607,0.174200,1.458933,0.174200,...,1.788000,1.0,2.821607,1.0,1.788000,1.0,60.369472,0.174200,60.369472,1.0
2030-10-07T22:00,-213.491169,0.0,-0.0,0.0,1.056800,0.153921,1.119091,0.153921,1.548800,0.153921,...,1.728533,1.0,2.812091,1.0,1.728533,1.0,59.572052,0.153921,59.572052,1.0


In [3]:
os.chdir(home_path)
os.chdir('Use_Case_2')
use_case_home_path = os.getcwd()
start = time.time()
df2 = f.co2_emissions(use_case_home_path)
print(f'time: {round(time.time() - start, 2)}')
df2

time: 121.07


,germany_grey_energy [kWh],germany_grey_energy [%],region-5_grey_energy [kWh],region-5_grey_energy [%],region-4_grey_energy [kWh],region-4_grey_energy [%],region-2_grey_energy [kWh],region-2_grey_energy [%],region-1_grey_energy [kWh],region-1_grey_energy [%],...,region-6-ec4-house-7_id-7-ev-non-commuter_grey_energy [kWh],region-6-ec4-house-7_id-7-ev-non-commuter_grey_energy [%],region-6-ec4-house-8_region-6-id-78-heatpump-loadprofile_grey_energy [kWh],region-6-ec4-house-8_region-6-id-78-heatpump-loadprofile_grey_energy [%],region-6-ec4-house-8_region-6-id-511-hh-2-household-electric-load_grey_energy [kWh],region-6-ec4-house-8_region-6-id-511-hh-2-household-electric-load_grey_energy [%],region-6-ec4-house-9_region-6-id-512-hh-2-household-electric-load_grey_energy [kWh],region-6-ec4-house-9_region-6-id-512-hh-2-household-electric-load_grey_energy [%],region-6-industry-electric-load_grey_energy [kWh],region-6-industry-electric-load_grey_energy [%]
2030-01-01T00:00,770.863572,0.738527,131.769877,1.000000,135.352697,1.0,274.617577,1.000000,-0.000000,0.0,...,0.004006,1.0,3.129416,1.0,1.098400,1.0,1.098400,1.0,64.715234,1.0
2030-01-01T01:00,808.200861,0.877856,66.101103,0.716575,137.453771,1.0,275.905834,1.000000,85.518285,1.0,...,0.017317,1.0,3.120415,1.0,1.038667,1.0,1.038667,1.0,62.774245,1.0
2030-01-01T02:00,767.327944,0.875751,73.737501,0.776882,133.911654,1.0,268.103002,1.000000,82.871609,1.0,...,0.011987,1.0,3.112027,1.0,1.055733,1.0,1.055733,1.0,63.878920,1.0
2030-01-01T03:00,806.917892,0.850986,132.011202,1.000000,136.005437,1.0,260.060515,1.000000,81.742299,1.0,...,0.005496,1.0,3.104372,1.0,1.932000,1.0,1.932000,1.0,64.737669,1.0
2030-01-01T04:00,920.101311,0.819207,141.280008,0.979593,129.365379,1.0,187.272410,0.803916,88.615231,1.0,...,0.004457,1.0,3.097578,1.0,1.860000,1.0,1.860000,1.0,65.167316,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-10-07T19:00,181.346512,0.186586,81.039561,1.000000,-0.000000,0.0,148.964844,1.000000,25.844908,1.0,...,0.099759,1.0,2.841004,1.0,1.088000,1.0,1.088000,1.0,60.644546,1.0
2030-10-07T20:00,161.209832,0.325583,-0.000000,0.000000,63.307778,1.0,90.218959,0.901131,0.000000,0.0,...,0.083408,1.0,2.831267,1.0,1.204000,1.0,1.204000,1.0,60.003041,1.0
2030-10-07T21:00,184.617216,0.192502,94.566021,1.000000,119.611786,1.0,-0.000000,0.000000,35.233065,1.0,...,0.069540,1.0,2.821607,1.0,1.788000,1.0,1.788000,1.0,60.369472,1.0
2030-10-07T22:00,203.386686,0.233204,120.221912,1.000000,-0.000000,0.0,147.884524,1.000000,13.294803,1.0,...,0.062291,1.0,2.812091,1.0,1.728533,1.0,1.728533,1.0,59.572052,1.0
